In [1]:
import sys
sys.path.append('../src')
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import tensorflow as tf
from gait.config import pd
from gait.utils import get_data_by_overlap_percent,get_overlap_data_all_sessions,  split_test_train_by_subjects, remove_invalid_data, get_overlap_data_all_sessions
from gait.training import train_lstm_model
from gait.evalution import save_history, save_test_history, save_accuracy_loss_figure, save_confusion_matrix_figure, compute_validations_predictions

In [2]:
overlapPercents = [80]
# overlapPercents = [0]
exclude_subjects = ['ddAeJA42PXvwthbW', 'nan', 'sUZBISq61Y7I5tqQ', 'LLZjAPTyj7muHsEf',
                    'cbOZWnI7s1y8oLD4', 'EUbKPOSQgjccjtvi', 'MMuX9YIh4NTbLZLM',
                    'PE8D53oX060qLbdX', 'ddAeJA42PXvwthbW', 'xYdtS1F8tDyjEIgN', '19AoxD1bgrDckd2p',
                    'wtyNo4LYaWXrkzA7' ]

for OVERLAP_PERCENT in overlapPercents:
    gpus = tf.config.list_logical_devices('GPU')
    strategy = tf.distribute.MirroredStrategy(gpus)
    with strategy.scope():
        X, y, subjects = get_overlap_data_all_sessions(OVERLAP_PERCENT)
        # REMOVE UNWANTED SUBJECTS
        print('unique subjects', np.unique(subjects))
        indexes = np.where(subjects == exclude_subjects)
        X = np.delete(X, indexes[0], axis=0)
        y = np.delete(y, indexes[0], axis=0)
        subjects = np.delete(subjects, indexes[0], axis=0)
        # END REMOVE UNWANTED SUBJECTS

        # AUGMENT DATA
        # resultant_acc = compute_resultant_acceleration(X)
        # resultant_gyro = compute_resultant_gyro(X)
        # resultant_angle = compute_resultant_angle(X)
        # resultant_acc = resultant_acc.reshape(
        #     resultant_acc.shape[0], resultant_acc.shape[1], 1)
        # resultant_gyro = resultant_gyro.reshape(
        #     resultant_gyro.shape[0], resultant_gyro.shape[1], 1)
        # resultant_angle = resultant_angle.reshape(
        #     resultant_angle.shape[0], resultant_angle.shape[1], 1)
        # X = np.concatenate((X, resultant_acc), axis=2)
        # X = np.concatenate((X, resultant_gyro), axis=2)
        # X = np.concatenate((X, resultant_angle), axis=2)
        # AUGMENT DATA END

        # print('UNIQUE subjects:', np.unique(subjects))
        X_train, X_test, y_train, y_test, y_train_non_encoded, y_test_non_encoded = split_test_train_by_subjects(
            X, y, subjects, train_percent=0.8)

        model, history = train_lstm_model(
            X_train, y_train, overlap_percent=OVERLAP_PERCENT, batch_size=128, epochs=1)
        # n_steps, n_length = 4, 32
        # X_test = X_test.reshape((X_test.shape[0], n_steps, n_length, 12))
        evaluation_history = model.evaluate(
            X_test, y_test, batch_size=128, verbose=1)

        save_history(history, OVERLAP_PERCENT)
        save_test_history(evaluation_history, OVERLAP_PERCENT)
        save_accuracy_loss_figure(history, OVERLAP_PERCENT)
        validations, predictions = compute_validations_predictions(
            model, X_test, y_test)
        save_confusion_matrix_figure(
            validations, predictions, OVERLAP_PERCENT, sns)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
unique subjects ['0DXZFsYOfgqsYe0Z' '0UXFxu10k0En1vDK' '19AoxD1bgrDckd2p'
 '1ZstYhEKzKKfGnMW' 'AlOXZjo87HHkXeaa' 'EUbKPOSQgjccjtvi'
 'I1TuySg6fbJ0xeXt' 'KNmC3LADpLyH5qJA' 'KPJOUYXzpivbLfsH'
 'LLZjAPTyj7muHsEf' 'LdH95bTaViXIdDBZ' 'MMuX9YIh4NTbLZLM'
 'NKdnm6cN3KKAx7R4' 'PE8D53oX060qLbdX' 'QFxq3HxMCbzs6bCt'
 'WAb2Ztvv2ux95P4Q' 'YbJVcb4vFFjKKlfe' 'bQsRwKxgJiXTzo6P'
 'c9EB3mOQckRCc8Bz' 'cbOZWnI7s1y8oLD4' 'csujYZktn88ftTTv'
 'ddAeJA42PXvwthbW' 'eqUcL0HkHZfDt0Xs' 'ewpnQDEzR8hTqPRY'
 'kPNKVxgIznp3Vskg' 'nan' 'psiuhPJmLIa2wC4u' 'qyP4saGxcV9UfeQI'
 'sUZBISq61Y7I5tqQ' 'wtyNo4LYaWXrkzA7' 'xYdtS1F8tDyjEIgN'
 'zOmNofnVfbR21Y7n']
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               44000     
                                                         

RuntimeError: Layer lstm:<class 'keras.layers.recurrent_v2.LSTM'> is not supported. You can quantize this layer by passing a `tfmot.quantization.keras.QuantizeConfig` instance to the `quantize_annotate_layer` API.